<a href="https://colab.research.google.com/github/It-HyunJin/OSS_Project/blob/Seeun/Depression__diary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy==1.23.1
!pip install kss

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-r9u0ymfs/kobert-tokenizer_a75c1e591d6d4872a6b5a4f3cbc20566
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-r9u0ymfs/kobert-tokenizer_a75c1e591d6d4872a6b5a4f3cbc20566
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import kss

In [5]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
device = torch.device("cpu")

In [7]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/final.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
chatbot_data.loc[(chatbot_data['value'] == "기쁨"), 'value'] = 0
chatbot_data.loc[(chatbot_data['value'] == "슬픔"), 'value'] = 1
chatbot_data.loc[(chatbot_data['value'] == "공포"), 'value'] = 2
chatbot_data.loc[(chatbot_data['value'] == "놀람"), 'value'] = 3
chatbot_data.loc[(chatbot_data['value'] == "분노"), 'value'] = 4
chatbot_data.loc[(chatbot_data['value'] == "불안"), 'value'] = 5

In [10]:
data_list = []
for q, label in zip(chatbot_data['text'], chatbot_data['value'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data)
print(data_list[:10])

[' 7로 나온다든데 아니었나', '3']
[['일은 왜 해도 해도 끝이 없을까? 화가 난다.', '4'], ['이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.', '4'], ['회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. ', '4'], ['직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.', '4'], ['얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.', '4'], ['직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.', '4'], ['성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.', '4'], ['퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.', '0'], ['졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.', '5'], ['요즘 직장생활이 너무 편하고 좋은 것 같아!', '0']]


In [11]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

In [12]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [13]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [14]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 6,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-33-c43874aa7363>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1302 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7388399839401245 train acc 0.203125
epoch 1 batch id 201 loss 0.5591802597045898 train acc 0.5936722636815921
epoch 1 batch id 401 loss 0.6887330412864685 train acc 0.7013715710723192
epoch 1 batch id 601 loss 0.6312181353569031 train acc 0.7326591098169717
epoch 1 batch id 801 loss 0.6126608848571777 train acc 0.7467228464419475
epoch 1 batch id 1001 loss 0.6032874584197998 train acc 0.753996003996004
epoch 1 batch id 1201 loss 0.6266276836395264 train acc 0.7592501040799334
epoch 1 train acc 0.7613933249174756


<ipython-input-33-c43874aa7363>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/326 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7909796779141104


  0%|          | 0/1302 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.11604782938957214 train acc 0.953125
epoch 5 batch id 201 loss 0.13808798789978027 train acc 0.9388215174129353
epoch 5 batch id 401 loss 0.08050431311130524 train acc 0.9443188902743143
epoch 5 batch id 601 loss 0.15617015957832336 train acc 0.947015391014975
epoch 5 batch id 801 loss 0.1731189340353012 train acc 0.9494577091136079
epoch 5 batch id 1001 loss 0.16315102577209473 train acc 0.9509865134865135
epoch 5 batch id 1201 loss 0.37683290243148804 train acc 0.9507832014987511
epoch 5 train acc 0.9511165310324542


  0%|          | 0/326 [00:00<?, ?it/s]

epoch 5 test acc 0.7906784070990359


In [18]:
%cd /content/drive/MyDrive/Final

/content/drive/MyDrive/Final


In [19]:
import os
os.chdir('/content/drive/MyDrive/Final')

# Load the entire model, mapping tensors to CPU
model1 = torch.load('6emotions_model.pt', map_location=torch.device('cpu'))

# Load state_dict, also mapping tensors to CPU
model1.load_state_dict(torch.load('6emotions_model_state_dict.pt', map_location=torch.device('cpu')))

# Load checkpoint, mapping tensors to CPU
checkpoint = torch.load('6emotions_all.tar', map_location=torch.device('cpu'))
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [47]:
def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨 혹은 중립이")
            elif np.argmax(logits) == 1:
                test_eval.append("우울이")
            elif np.argmax(logits) == 2:
                test_eval.append("공포가")
            elif np.argmax(logits) == 3:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 4:
                test_eval.append("분노가")
            elif np.argmax(logits) == 5:
                test_eval.append("불안이")


        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
    return np.argmax(logits)

In [48]:
def depression_predict(sentence):
  text = list()
  depression = 0
  for sent in kss.split_sentences(sentence):
    print(sent)
    text.append(sent)
    i = predict(sent)
    if (i == 1):
      depression += 1
    elif (i == 5 or i == 4 or i == 3):
      depression += 0.3
  print(str(round(depression/len(text)*100,1))+"%로 우울합니다.")

In [57]:
s = "~"
depression_predict(s)

오랜만에 꿀잠을 잤구만
>> 입력하신 내용에서 기쁨 혹은 중립이 느껴집니다.
그래서 일어나자마자 만두 신라면 먹음.
>> 입력하신 내용에서 기쁨 혹은 중립이 느껴집니다.
그리고 유튜브도 보다가 그냥 놀았음.
>> 입력하신 내용에서 놀람이 느껴집니다.
세빈이는 자길래 옆에서 그냥 학습 돌려놓고 여추반 봤음!
>> 입력하신 내용에서 놀람이 느껴집니다.
요즘 ai 학습시키는게 기말과제라 열심히 하는 중임ㅠㅠ 휴 힘들다.
>> 입력하신 내용에서 공포가 느껴집니다.
그래도 성과가 나오니까 재밌긴함.
>> 입력하신 내용에서 기쁨 혹은 중립이 느껴집니다.
이제 진짜 종강 막바지라 놀고싶당~
>> 입력하신 내용에서 기쁨 혹은 중립이 느껴집니다.
8.6%로 우울합니다.
